```python
what I learned:
- map_dict({1:'online'})
- str.strptime(pl.Utf8, format=)
- str.n_chars
- dt.quarter()

- .rename({'name':'name_importer','region':'region_importer'})

all dt functions:
- year(): Extract the year from a datetime column.
- month(): Extract the month from a datetime column.
- day(): Extract the day from a datetime column.
- hour(): Extract the hour from a datetime column.
- minute(): Extract the minute from a datetime column.
- second(): Extract the second from a datetime column.
- nanosecond(): Extract the nanosecond from a datetime column.
- weekday(): Extract the weekday from a datetime column.
- week(): Extract the week from a datetime column.
- quarter(): Extract the quarter from a datetime column.
- yearweek(): Extract the yearweek from a datetime column.

```

In [80]:
import polars as pl

quarterly_targets_file = 'https://drive.google.com/u/0/uc?id=1YNg4R6Ud_Dxi108VJvxndlQNe5Rn9IdI&export=download'
transactions_file = 'https://drive.google.com/u/0/uc?id=10EYKJ989gReQJhOGHx9vK7iPOHii-g_q&export=download'
pl.Config.set_fmt_str_lengths(50)

targets_df=pl.read_csv(quarterly_targets_file)
transactions_df=pl.read_csv(transactions_file, try_parse_dates=True)


In [81]:
# targets dataframe general view
targets_df.head(1)

Online or In-Person,Q1,Q2,Q3,Q4
str,i64,i64,i64,i64
"""Online""",72500,70000,60000,60000


In [82]:
# tranaction dataframe general view
transactions_df.head(1)

Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date
str,i64,i64,i64,datetime[μs]
"""DTB-716-679-576""",1448,100001,2,2023-03-20 00:00:00


In [83]:
# ERD diagram: length of transaction code
transactions_df.with_columns(
    pl.col('Transaction Code').str.n_chars().max().alias('max_chars'),
    pl.col('Transaction Code').str.n_chars().min().alias('min_chars')
)

Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date,max_chars,min_chars
str,i64,i64,i64,datetime[μs],u32,u32
"""DTB-716-679-576""",1448,100001,2,2023-03-20 00:00:00,15,14
"""DS-795-814-303""",7839,100001,2,2023-11-15 00:00:00,15,14
"""DSB-807-592-406""",5520,100005,1,2023-07-14 00:00:00,15,14
"""DS-367-545-264""",7957,100007,2,2023-08-18 00:00:00,15,14
"""DSB-474-374-857""",5375,100000,2,2023-08-26 00:00:00,15,14
"""DSB-448-546-348""",4525,100009,1,2023-05-27 00:00:00,15,14
"""DS-213-969-866""",2321,100007,2,2023-01-04 00:00:00,15,14
"""DS-341-615-729""",1967,100004,1,2023-01-22 00:00:00,15,14
"""DTB-200-951-620""",8105,100003,2,2023-01-10 00:00:00,15,14


In [84]:
# Split transaction code into bank (AAA - 3 digits) and transaction number
transactions_df=transactions_df.with_columns(
    pl.col('Transaction Code').str.splitn('-',2)
).unnest('Transaction Code')

# Map 1:'Online' and 2:'In-Person'
transactions_df=transactions_df.with_columns(
    pl.col('Online or In-Person').map_dict({1:'Online',2:'In-Person'})
)

# filter out all banks except BSD
transactions_df=transactions_df.filter(
    pl.col('field_0') == 'DSB'
)

In [85]:
# Convert Transaction Date to quarter
transaction_df = transactions_df.with_columns(
    pl.col('Transaction Date').dt.quarter()
)

# Rename all columns/fields as needed
transaction_df=transaction_df.with_columns(
    pl.col('Transaction Date').cast(pl.Utf8).alias('Quarter'),
    pl.col('field_0').alias('Bank'),
    pl.col('field_1').alias('Transaction Number')
).drop(['Transaction Date','field_0','field_1'])

# Reorder Dataframe
transaction_df=transaction_df.select([
    'Bank',
    'Transaction Number',
    'Customer Code',
    'Quarter',
    'Online or In-Person',
    'Value'])

In [86]:
transaction_df.head(1)

Bank,Transaction Number,Customer Code,Quarter,Online or In-Person,Value
str,str,i64,str,str,i64
"""DSB""","""807-592-406""",100005,"""3""","""Online""",5520


In [87]:
# Groupby data by Quarter and Online or in person and aggregate Values
transaction_df_final=transaction_df.groupby(
    pl.col(['Quarter','Online or In-Person'])
    ).agg(pl.col('Value').sum()
          ).sort('Quarter')

In [88]:
transaction_df_final.head(20)

Quarter,Online or In-Person,Value
str,str,i64
"""1""","""In-Person""",77576
"""1""","""Online""",74562
"""2""","""Online""",69325
"""2""","""In-Person""",70634
"""3""","""Online""",59072
"""3""","""In-Person""",74189
"""4""","""Online""",61908
"""4""","""In-Person""",43223


In [89]:
targets_df.head()

Online or In-Person,Q1,Q2,Q3,Q4
str,i64,i64,i64,i64
"""Online""",72500,70000,60000,60000
"""In-Person""",75000,70000,70000,60000


In [90]:
melted_targets_df=targets_df.melt(
    id_vars='Online or In-Person',
    value_vars=['Q1','Q2','Q3','Q4']
)

# Melted_targets_df=melted_targets_df.select(pl.col(['variable','value','Online or In-Person']))
melted_targets_df=melted_targets_df.with_columns(
    pl.col('variable').map_dict({'Q1':'1','Q2':'2','Q3':'3','Q4':'4'})
)

# Rename columns
melted_targets_df=melted_targets_df.with_columns(
    pl.col('variable').alias('Quarter'),
    pl.col('value').alias('Quarterly Targets')
).drop(['variable','value'])

# Head
melted_targets_df.head(10)

# (
#     sales_pv
#     .melt(
#         id_vars="date",
#         value_vars=["Mountain","Road"]
#     )
# )



Online or In-Person,Quarter,Quarterly Targets
str,str,i64
"""Online""","""1""",72500
"""In-Person""","""1""",75000
"""Online""","""2""",70000
"""In-Person""","""2""",70000
"""Online""","""3""",60000
"""In-Person""","""3""",70000
"""Online""","""4""",60000
"""In-Person""","""4""",60000


In [91]:
# Join transaction_df and targets_df

final_table=transaction_df_final.join(
    melted_targets_df,
    left_on=['Quarter','Online or In-Person'],
    right_on=['Quarter','Online or In-Person'],
    how='inner'
).select(['Online or In-Person','Quarter','Value','Quarterly Targets'])


# final_table=transaction_df_final.join(
#     melted_targets_df,
#     left_on=['Quarter','Online or In-Person'],
#     right_on=['Quarter','Online or In-Person'],
#     how='inner'
# ).drop(['Online or In-Person_right']
#        ).select(['Online or In-Person','Quarter','Value','Quarterly Targets'])

final_table=final_table.with_columns(
    (pl.col('Value') - pl.col('Quarterly Targets')).alias('Variance')
    )




In [92]:
final_table.head(20)

Online or In-Person,Quarter,Value,Quarterly Targets,Variance
str,str,i64,i64,i64
"""Online""","""1""",74562,72500,2062
"""In-Person""","""1""",77576,75000,2576
"""Online""","""2""",69325,70000,-675
"""In-Person""","""2""",70634,70000,634
"""Online""","""3""",59072,60000,-928
"""In-Person""","""3""",74189,70000,4189
"""Online""","""4""",61908,60000,1908
"""In-Person""","""4""",43223,60000,-16777


In [95]:
# (
#     dfCITES
#     .join(
#         dfISO
#         ,left_on='Importer'
#         ,right_on='alpha-2'
#         ,how='left'
#     ).rename(
#         {
#             'name':'name_importer'
#             ,'region':'region_importer'
#         }
#     )
# )